In [6]:
import pathlib
import pandas as pd
import api_setup
import spotipy
REPO_ROOT = pathlib.Path.cwd().parent

In [7]:
# API Auth
env_vars = api_setup.parse_api_kvs(pathlib.Path.cwd().parent / "api-keys")
auth_manager = spotipy.SpotifyClientCredentials(env_vars['client_id'], env_vars['client_secret'])
spotify = spotipy.Spotify(client_credentials_manager=auth_manager, backoff_factor=2)

In [8]:
from typing import List

In [9]:
def get_songs_from_playlist(spotify_client: spotipy.Spotify, playlist_uri: str) -> List[str]:
    """
    Return a list of strings of the URIs of the tracks in this playlist.
    """
    tracks_json = spotify_client.playlist_items(playlist_uri)
    return [track['track']['uri'][14:] for track in tracks_json['items']]

get_songs_from_playlist(spotify, "spotify:playlist:76S2ElS2cyzY624wGBGKpB")

['7yl0ItOwlAnALSctbUiavO',
 '4LpUpiYoZ2M3Z1kmhn4EQo',
 '1OuN92HcVG6NVpWbeESNB3',
 '44SO1hMPfH9xUvmI7bjhou',
 '58e7V70Em6FABOiln4jNoZ',
 '2Xl2dfsBQYaPP5I2viTVr9',
 '7L4G39PVgMfaeHRyi1ML7y',
 '1GpZofCtuWj4adPQLqpeFw',
 '3q6ygCZID0OKj6MUxInB48']

In [10]:
# async api calls
async def get_audio_features(session: ClientSession, uri: str) -> dict:
    """
    Return the audio features of the song with the given uri.
    """
    endpoint = f"https://api.spotify.com/v1/audio-features/{uri}"

    async with session.get(endpoint) as response:
        response = await(response.json())
    return uri, response

def get_header_with_token(client_id: str, client_secret: str):
    creds = f"{env_vars['client_id']}:{env_vars['client_secret']}"
    creds_b64 = base64.b64encode(creds.encode())
    headers= {"Authorization": f"Basic {creds_b64.decode()}"}
    data= {"grant_type": "client_credentials"}
    token = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)
    token = token.json()['access_token']
    return {"Accept": "application/json", "Content-Type": "application/json", "Authorization": f"Bearer {token}"}


async def featurize_song_list(client_id:str, client_secret: str, song_uris: List[str]) -> dict:
    request_headers = get_header_with_token(client_id, client_secret)
    async with ClientSession(headers=request_headers) as session:
        tasks = [asyncio.ensure_future(get_audio_features(session, uri)) for uri in song_uris]
        features = await(asyncio.gather(*tasks))
    return features

async def featurize_playlist(spotify_client: spotipy.Spotify, client_id:str, client_secret: str, playlist_uri: str) -> dict:
    song_uris = get_songs_from_playlist(spotify_client, playlist_uri)
    playlist_song_features = await(featurize_song_list(client_id, client_secret, song_uris))
    return playlist_song_features

features = await(featurize_playlist(spotify, env_vars['client_id'], env_vars['client_secret'], "spotify:playlist:76S2ElS2cyzY624wGBGKpB"))
features

NameError: name 'ClientSession' is not defined